<a href="https://colab.research.google.com/github/kapibara3chi/NLPTesForGC/blob/main/NLPTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [1]:
!pip  install -U ginza ja_ginza_electra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 10.5 MB/s 
     |████████████████████████████████| 2.2 MB 33.0 MB/s 
     |████████████████████████████████| 53 kB 496 kB/s 
     |████████████████████████████████| 283 kB 22.6 MB/s 
     |████████████████████████████████| 4.7 MB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 56.7 MB/s 
     |████████████████████████████████| 7.6 MB 39.2 MB/s 
     |████████████████████████████████| 182 kB 43.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.2 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=fc6835d3b1f8f0929906edd350f739877acc354c7aac57650444a66a3f66841a
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
  Created wheel for sudachitra: filename=SudachiTra-0.1.7-py3-none-any.whl size=266095 sha256=771a

# main

In [11]:
import ginza
from spacy import displacy
import spacy
class DependencyAnalysis:
    def __init__(self):
        self.nlp = spacy.load('ja_ginza_electra')
        self.indices = []

    def get_analysis(self, text):
        """係り受け解析"""
        doc = self.nlp(text)

        token_head_list = []

        for sent in doc.sents:
            for token in sent:
                token_head_list.append(
                    {"i": token.i, "orth": token.orth_, "base": token.lemma_,
                     "head": token.head.i, "dep": token.dep_})

        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

        return token_head_list
    def get(self, text, target):
      # 係り受け解析
      self.depend_indices = self.get_analysis(text)
      print(self.depend_indices)

    def get_words(self,  m, head):
        """対象の領域の抽出"""
        
        if m["head"] == head :
            for n in self.depend_indices:
                self.get_words(m=n, head=m["i"])
                            
            self.indices.append(m["i"])
        
        return
    def get(self, text, target):
        
        # 係り受け解析
        self.depend_indices = self.get_analysis(text)
        #print(self.depend_indices)
                
        # 係り元のインデックスを取得
        head = self.depend_indices[target]["head"]

        print(self.depend_indices)
        # 係り元のかかり先を探索
        self.indices.append(head)
        for m in self.depend_indices:
            if m["head"] == head and m["dep"] != "ROOT":
                self.get_words(m, head=head)

        clause = ""
        for clause_num in sorted(self.indices):
            clause = clause + str(self.depend_indices[clause_num]["orth"])
            
        return clause

if __name__ == '__main__':
  dependency = DependencyAnalysis()
  txt="下流あごをだすと分離は改善するが、コンタミが悪化する"
  # ret=dependency.get_analysis("下流あごをだすと分離は改善するが、コンタミが悪化する")
  # for i in ret:
    # print(i)
  dependency.get(txt,0)


/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


{'i': 0, 'orth': '下流', 'base': '下流', 'head': 1, 'dep': 'compound'}
{'i': 1, 'orth': 'あご', 'base': 'あご', 'head': 3, 'dep': 'obj'}
{'i': 2, 'orth': 'を', 'base': 'を', 'head': 1, 'dep': 'case'}
{'i': 3, 'orth': 'だす', 'base': 'だす', 'head': 7, 'dep': 'advcl'}
{'i': 4, 'orth': 'と', 'base': 'と', 'head': 3, 'dep': 'mark'}
{'i': 5, 'orth': '分離', 'base': '分離', 'head': 7, 'dep': 'nsubj'}
{'i': 6, 'orth': 'は', 'base': 'は', 'head': 5, 'dep': 'case'}
{'i': 7, 'orth': '改善', 'base': '改善', 'head': 13, 'dep': 'advcl'}
{'i': 8, 'orth': 'する', 'base': 'する', 'head': 7, 'dep': 'aux'}
{'i': 9, 'orth': 'が', 'base': 'が', 'head': 7, 'dep': 'mark'}
{'i': 10, 'orth': '、', 'base': '、', 'head': 7, 'dep': 'punct'}
{'i': 11, 'orth': 'コンタミ', 'base': 'コンタミ', 'head': 13, 'dep': 'nsubj'}
{'i': 12, 'orth': 'が', 'base': 'が', 'head': 11, 'dep': 'case'}
{'i': 13, 'orth': '悪化', 'base': '悪化', 'head': 13, 'dep': 'ROOT'}
{'i': 14, 'orth': 'する', 'base': 'する', 'head': 13, 'dep': 'aux'}


[{'i': 0, 'orth': '下流', 'base': '下流', 'head': 1, 'dep': 'compound'}, {'i': 1, 'orth': 'あご', 'base': 'あご', 'head': 3, 'dep': 'obj'}, {'i': 2, 'orth': 'を', 'base': 'を', 'head': 1, 'dep': 'case'}, {'i': 3, 'orth': 'だす', 'base': 'だす', 'head': 7, 'dep': 'advcl'}, {'i': 4, 'orth': 'と', 'base': 'と', 'head': 3, 'dep': 'mark'}, {'i': 5, 'orth': '分離', 'base': '分離', 'head': 7, 'dep': 'nsubj'}, {'i': 6, 'orth': 'は', 'base': 'は', 'head': 5, 'dep': 'case'}, {'i': 7, 'orth': '改善', 'base': '改善', 'head': 13, 'dep': 'advcl'}, {'i': 8, 'orth': 'する', 'base': 'する', 'head': 7, 'dep': 'aux'}, {'i': 9, 'orth': 'が', 'base': 'が', 'head': 7, 'dep': 'mark'}, {'i': 10, 'orth': '、', 'base': '、', 'head': 7, 'dep': 'punct'}, {'i': 11, 'orth': 'コンタミ', 'base': 'コンタミ', 'head': 13, 'dep': 'nsubj'}, {'i': 12, 'orth': 'が', 'base': 'が', 'head': 11, 'dep': 'case'}, {'i': 13, 'orth': '悪化', 'base': '悪化', 'head': 13, 'dep': 'ROOT'}, {'i': 14, 'orth': 'する', 'base': 'する', 'head': 13, 'dep': 'aux'}]
